This notebook will use the robin_stocks.robinhood.export functions which will write all options orders to a csv file and parse them to calculate the total cost of the options orders.  It will also get all the events for the options orders and calculate the total cost of the options events.  The sum of the options orders and events costs will give the total cost of the options.

Install the required libraries

In [1]:
%pip install robin_stocks pandas numpy ipykernel


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Log on to the robinhood API endpoint, and create the output directory if it doesn't exist

In [2]:
import robin_stocks.robinhood as r
import pandas as pd
import os  
# Prompt for email address
email = input("Please enter your Robinhood email address: ")

login = r.login(email)

# Ensure the output directory exists
output_dir = '../output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir) 


Get all the options orders using the export_completed_option_orders function.

In [3]:
# # Get all option orders
r.export_completed_option_orders('/', file_name='../output/options_output.csv')

Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...


This groups spread order by order_created_at and then calculates total cost of all the option orders based on debit and credit direction.  Spreads orders returned from the api are listed multiple times. For example if there is a spread order with 2 buys and 2 sells, a total of 4 orders will be listed with the total cost of the spread, not the actual respective order leg costs.  So we group the spreads by order_created_date and pull the spread order cost once to get accurate costs.

There is some code commented out here as part of an attempt to pair the opening and closing orders, and may continue that at some point but for now this appears to correctly calculate costs 

In [4]:
import pandas as pd

# Load your spreadsheet
df = pd.read_csv('../output/options_output.csv')

# Convert 'order_created_at' to datetime
df['order_created_at'] = pd.to_datetime(df['order_created_at'])
df['order_created_at'] = df['order_created_at'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Filter for the year 2024
#df = df[df['order_created_at'].str.startswith('2024')]

# Select only the columns we want
df = df[['order_created_at', 'chain_symbol', 'expiration_date', 
         'strike_price', 'option_type', 'direction', 'order_quantity', 
         'order_type', 'opening_strategy', 'closing_strategy', 'price', 'processed_quantity']]

# Group by 'order_created_at' and aggregate
aggregated_df = df.groupby('order_created_at').agg({
    'chain_symbol': 'first',
    'expiration_date': 'first',
    'strike_price': lambda x: list(sorted(x, key=abs, reverse=True)),  # Create a list of strike price
    'option_type': 'first',
    'direction': 'first',
    'opening_strategy': 'first',
    'closing_strategy': 'first',
    'price': 'first',
    'order_quantity': 'first'
}).reset_index()

# Sort the DataFrame
aggregated_df = aggregated_df.sort_values(by=['chain_symbol', 'expiration_date'])

# Calculate cost
aggregated_df['cost'] = aggregated_df['price'] * aggregated_df['order_quantity'] * 100

# Adjust the cost based on the direction
aggregated_df['cost'] = aggregated_df.apply(
    lambda x: -x['cost'] if x['direction'] == 'debit' else x['cost'],
    axis=1
)      

total_option_cost = aggregated_df['cost'].sum()
print(f"Total option cost: ${total_option_cost:.2f}")
print(aggregated_df['order_created_at'].count())
print(aggregated_df['order_quantity'].sum())

# Save the results
aggregated_df.to_csv('../output/options_orders_parsed.csv', index=False)

# Count rows where 'opening_strategy' is not None
total_opening_strategy_count = aggregated_df['opening_strategy'].notna().sum()
total_closing_strategy_count = aggregated_df['closing_strategy'].notna().sum()


# # Count rows where 'closing_strategy' contains 'spread' or 'iron'
# spread_closing_count = df['closing_strategy'].str.contains('spread|iron', case=False, na=False).sum()

# # Count rows where 'opening_strategy' contains 'spread' or 'iron'
# spread_opening_count = df['opening_strategy'].str.contains('spread|iron', case=False, na=False).sum()

# # Count rows where 'closing_strategy' contains 'spread' or 'iron'
# non_spread_closing_count = total_closing_strategy_count - spread_closing_count

# # Count rows where 'opening_strategy' contains 'spread' or 'iron'
# non_spread_opening_count = total_opening_strategy_count - spread_opening_count
# Calculate total order quantity for opening_strategy
total_opening_quantity = aggregated_df['order_quantity'][aggregated_df['opening_strategy'].notna()].sum()
print(f"Total opening strategy quantity: {total_opening_quantity}")

# Calculate total order quantity for closing_strategy
total_closing_quantity = aggregated_df['order_quantity'][aggregated_df['closing_strategy'].notna()].sum()
print(f"Total closing strategy quantity: {total_closing_quantity}")


#Print the counts
print(f"Opening strategy rows: {total_opening_strategy_count}")
print(f"Closing strategy rows: {total_closing_strategy_count}")
# print(f"Opening strategy rows with 'spread' or 'iron': {spread_opening_count}")
# print(f"Closing strategy rows with 'spread' or 'iron': {spread_closing_count}")
# print(f"Opening strategy rows not with 'spread' or 'iron': {non_spread_opening_count}")
# print(f"Closing strategy rows not with 'spread' or 'iron': {non_spread_closing_count}")

Total option cost: $-3247.00
878
1625.0
Total opening strategy quantity: 1008.0
Total closing strategy quantity: 618.0
Opening strategy rows: 544
Closing strategy rows: 335


In [59]:
# ... existing code ...
import pandas as pd
import json  # Use json to parse the string representation of lists/dictionaries

unique_symbols = aggregated_df['chain_symbol'].unique()
total_option_event_cost = 0  # Initialize total cash amount

# Create a list to store event data
event_data = []
raw_event_data = []

for symbol in unique_symbols:
    events = r.get_events(symbol)  # Fetch events for the symbol
    for event in events:
        # Ignore rows with total_cash_amount = 0
        if float(event['total_cash_amount']) == 0 :
            continue
        #print(event)
        raw_event_data.append(event)
        # Append event details to the list
        event_data.append({
            'created_at': event['created_at'],
            'chain_symbol': symbol,
            'direction': event['direction'],
            'quantity': event['quantity'],
            'total_cash_amount': event['total_cash_amount'],
            'state': event['state'],
            'underlying_price': event['underlying_price']
        })
        if event['direction'] == 'credit':
            total_option_event_cost += float(event['total_cash_amount'])  # Add for exercise
        elif event['direction'] == 'debit':
            total_option_event_cost -= float(event['total_cash_amount'])  # Subtract for assignment
        # if event['type'] == 'exercise':
        #     total_option_event_cost += float(event['total_cash_amount'])  # Add for exercise
        # elif event['type'] == 'assignment':
        #     total_option_event_cost -= float(event['total_cash_amount']) 
        
# Create a DataFrame from the event data
events_df = pd.DataFrame(event_data)

raw_events_df = pd.DataFrame(raw_event_data)
raw_events_df.to_csv('../output/option_events_raw.csv', index=True)
# Save the events DataFrame to a new CSV file
events_df.to_csv('../output/option_events.csv', index=True)


# Print the total cash amount
print(f"Total cash amount: {total_option_event_cost}")


Total cash amount: -36462.64000000001


In [70]:
# ... existing code ...
import pandas as pd
import json  # Use json to parse the string representation of lists/dictionaries

unique_symbols = aggregated_df['chain_symbol'].unique()
total_option_credits = 0
total_option_debits = 0
total_option_exercise = 0 
total_option_assignment = 0  

# Read the raw event data correctly
raw_event_data = pd.read_csv('../output/option_events_raw.csv')


# Create a list to store event data
event_data = []

# Iterate over the DataFrame rows
for index, event in raw_event_data.iterrows():  
    event_data.append({
        # 'symbol': event['equity_components']['symbol'],
        # 'price': event['equity_components']['price'],
        # 'quantity': event['equity_components']['quantity'],
        # 'side': event['equity_components']['side'],
        'created_at': event['created_at'],
        'direction': event['direction'],
        'quantity': event['quantity'],
        'total_cash_amount': event['total_cash_amount'],
        'state': event['state'],
        'underlying_price': event['underlying_price']
    })
    if event['direction'] == 'credit' and event['source_ref_id'] :
        total_option_credits += float(event['total_cash_amount'])  # Add for exercise
    elif event['direction'] == 'debit' and event['source_ref_id'] != '' :
        total_option_debits -= float(event['total_cash_amount'])  # Subtract for assignment
    
    if event['type'] == 'exercise':
        total_option_exercise += float(event['total_cash_amount'])  # Add for exercise
    elif event['type'] == 'assignment':
        total_option_assignment -= float(event['total_cash_amount']) 
    
# Create a DataFrame from the event data
events_df = pd.DataFrame(event_data)
# Save the events DataFrame to a new CSV file
events_df.to_csv('../output/option_events.csv', index=True)
# Print the total cash amount
print(f"Total option events {len(events_df)}")
print(f"Total option event credits: {total_option_credits}")
print(f"Total option event debits: {total_option_debits}")
print(f"Total options exercised: {total_option_exercise}")
print(f"Total options assigned: {total_option_assignment}")

print(f" exercise + assignment {total_option_exercise + total_option_assignment}")
print(f" credits + debits {total_option_credits + total_option_debits}")

print(f" exercised - credits {total_option_exercise - total_option_credits}")
print(f" assigned - debits { total_option_assignment - total_option_debits}")

total_option_event_cost = total_option_exercise - total_option_credits


Total option events 52
Total option event credits: 7529537.360000001
Total option event debits: -7566000.0
Total options exercised: 7540137.360000001
Total options assigned: -7555400.0
 exercise + assignment -15262.639999998733
 credits + debits -36462.63999999873
 exercised - credits 10600.0
 assigned - debits 10600.0


In [72]:
total =  total_option_cost + total_option_event_cost
print(total)

7353.0
